In [49]:
import os
import json
import praw
import time
import pandas as pd
from datetime import datetime

# API credentials 

reddit = praw.Reddit(
    client_id='Y5DdpLJDcxJPSAT7Vvmo-A',
    client_secret='KZ83gqe2B-C3uJwlcjEI4e1gR_gpPw',
    user_agent='test by u/Inner-Astronomer3735'
)

# constants 
max_depth = 45
max_comments = 30000

import time

import time

def wait_for_rate_limit():
    """ 
    Pauses execution if remaining requests are below 10 and waits until rate limit reset.
    Rate limit resets at 10 minute intervals. 
    """

    remaining = reddit.auth.limits['remaining']

    if remaining >= 10:
        return  # No need to wait

    # First sleep for the estimated reset time
    seconds_until_reset = reddit.auth.limits['reset_timestamp'] + 15 - time.time()

    if seconds_until_reset > 0:
        time_now = datetime.now().strftime("time %H:%M:%S")
        print(f'Sleeping for {seconds_until_reset:.2f} seconds until rate limit reset ({time_now}).')
        time.sleep(seconds_until_reset)

    # Generic API call to refresh "remaining" count.
    list(reddit.subreddits.popular(limit=1))

    if reddit.auth.limits['remaining'] < 20:
        print("Something unexpected happened when waiting for rate limit.")
        raise Exception('Rate limit did not reset as expected.')
    
    time_now = datetime.now().strftime("time %H:%M:%S")
    print(f"Resumed: remaining requests = {reddit.auth.limits['remaining']}. ({time_now})")



def fetch_comments(comment_forest, depth=0):

    wait_for_rate_limit()

    if depth > max_depth:
        return []
    
    comment_trees = []

    # loop through root nodes
    for comment in comment_forest:
        
        # When there's too many comments the API collaposes some of them into MoreComments objects
        # This bit of code is to expand those and get the comments. 

        if isinstance(comment, praw.models.MoreComments):

            wait_for_rate_limit()

            more = comment.comments()
            comment_trees.extend(fetch_comments(more, depth))

            continue

        dictionary = {
                    "author": str(comment.author),
                    "body": comment.body,
                    "score": comment.score,
                    "replies": fetch_comments(comment.replies, depth+1)
                }
        
        comment_trees.append(dictionary)

    return comment_trees

# The input is a post object (not a post_id like it used to be, but the actual post object)
# Timestamp is used as part of the directory title, for organization
def fetch_post(post, rank, timestamp, target_base_directory): 

    wait_for_rate_limit()

    comments_thread = post.comments #TODO rename to comment forest

    comments_processed = fetch_comments(comments_thread)

    # create file path
    subreddit_folder = os.path.join(target_base_directory, post.subreddit.display_name)
    timestamp_folder = os.path.join(subreddit_folder, timestamp)

    if not os.path.exists(timestamp_folder):
        os.makedirs(timestamp_folder)

    post_file = os.path.join(timestamp_folder,f'{post.id}.json')


    # gather post data
    post_data = {
                "title": post.title,
                "author": str(post.author),
                "subreddit": post.subreddit.display_name,
                "rank" : rank,
                "score": post.score,
                "upvote_ratio": post.upvote_ratio,
                "num_comments (reported by reddit)": post.num_comments,
                "url": post.url,
                "id": post.id,
                "selftext": post.selftext,
                "comments": comments_processed
            }
    
    with open(post_file, "w", encoding="utf-8") as file:
        json.dump(post_data, file, indent=4)
        # print(f'Saved {post.id}.json')

def fetch_subreddit(subreddit_name, n_posts, target_base_directory = 'data', max_runtime = 3000, print_frequency = 10): 

    skipped = 0
    downloaded = 0

    subreddit = reddit.subreddit(subreddit_name)
    
    # timestamp for directory name. (
    # We freeze this timestamp as soon as we run the function 
    # so that it doesn't change the folder name even if this takes time to run.
    # I.e., this is the stamp of when we began this scraping run. (any individual file was probably downloaded later).

    timestamp = datetime.now().strftime("date_%m-%d-%Y_time_%H-%M")

    # start time tracking (this is to interrupt the function if it runs for too long)
    start_time = time.time()

    # TODO: Do we want top or hot? 
    
    for rank, post in enumerate(subreddit.hot(limit=n_posts)):

        if time.time() - start_time > max_runtime:
            print(f"Stopping fetch for {subreddit_name}, exceeded max runtime.")
            break

        if post.num_comments < max_comments:
            
            # print(f'Fetching post {post.id} (rank = {rank}, num_comments = {post.num_comments}).')
            fetch_post(post, rank, timestamp, target_base_directory)
            downloaded += 1
            if downloaded % print_frequency == 1: 
                time_now = datetime.now().strftime("time %H:%M")
                print(f'Downloaded {downloaded} posts from {subreddit} so far ({time_now}).')

        else: 
            print(f'Skipped post with id {post.id} because it has {post.num_comments} comments. (max_comments set to {max_comments}).')
            skipped += 1
            print(f'Skipped count: {skipped}')
    time_now = datetime.now().strftime("time %H:%M")
    print(f'Downloaded {downloaded} posts from {subreddit} in total ({time_now}).')
    return downloaded

# Scraping run 1 (500 posts from 15 subreddits)

In [ ]:
# how many posts from each subreddit. 
n = 500

# Define three separate lists for each category
left_leaning = [
    #'communism',
    'socialism',
    #'LateStageCapitalism',
    'Anarchism',
    #'Marxism',
    'progressive'
]

right_leaning = [
    'Conservative',
    'Libertarian',
    'Anarcho_Capitalism',
    'Republican',
    'AskTrumpSupporters', # is this actually right leaning? might have a lot of "passers-by"
    'RedState'
]

uncontroversial_or_mixed = [
    'Politics',
    'news',
    'MadeMeSmile',
    'DogTraining',
    'ADHD',
    'stopdrinking',
    'aww',
    'wholesomememes'
]

# Combine all lists into one
all_subreddits = left_leaning + right_leaning + uncontroversial_or_mixed

# start time of scraping (for naming exceptions dataframe later)
scrape_start_time = datetime.now()

# dictionary to collect exceptions data
exceptions_data = {
    "exception_type": [],
    "exception_message": [],
    "subreddit": []
}

exception_counter = 0

for subreddit in all_subreddits:
    try:
        fetch_subreddit(subreddit, n)
    except Exception as e:
        exception_counter += 1

        exceptions_data["exception_type"].append(type(e).__name__)
        exceptions_data["exception_message"].append(str(e))
        exceptions_data["subreddit"].append(subreddit)

        if exception_counter % 3 == 0: print(f'{exception_counter} exceptions so far. Latest: {str(e)}, at subreddit = {subreddit}')

#if there actually were some exceptions, save the log:
if exception_counter: 
    exceptions_df = pd.DataFrame(exceptions_data)
    timestamp_str = scrape_start_time.strftime('%Y-%m-%d_%H-%M-%S')
    filename = f"exceptions_{timestamp_str}.csv"
    exceptions_df.to_csv(filename, index=False)
else:
    print("No exceptions raised!")


Downloaded 1 posts from socialism so far (time 17:08).
Downloaded 11 posts from socialism so far (time 17:08).
Downloaded 21 posts from socialism so far (time 17:08).
Downloaded 31 posts from socialism so far (time 17:08).
Downloaded 41 posts from socialism so far (time 17:08).
Downloaded 51 posts from socialism so far (time 17:08).
Downloaded 61 posts from socialism so far (time 17:08).
Downloaded 71 posts from socialism so far (time 17:08).
Downloaded 81 posts from socialism so far (time 17:08).
Downloaded 91 posts from socialism so far (time 17:08).
Downloaded 101 posts from socialism so far (time 17:08).
Downloaded 111 posts from socialism so far (time 17:09).
Downloaded 121 posts from socialism so far (time 17:09).
Downloaded 131 posts from socialism so far (time 17:09).
Downloaded 141 posts from socialism so far (time 17:09).
Downloaded 151 posts from socialism so far (time 17:09).
Downloaded 161 posts from socialism so far (time 17:09).
Downloaded 171 posts from socialism so far

# Scraping run 2 (20 posts from 300 top subreddits).

In [ ]:
# start time of scraping (for naming exceptions dataframe later)
scrape_start_time = datetime.now()

# dictionary to collect exceptions data
exceptions_data = {
    "exception_type": [],
    "exception_message": [],
    "subreddit": []
}

# dictionary to collect total downloads data
meta_data = {
    "subreddit": [],
    "total_downloaded_posts": [],
}

exception_counter = 0

# top_subreddits = [sub.display_name for sub in reddit.subreddits.popular(limit=300)]
top_subreddits = ['wallstreetbets', 'AskReddit']

for subreddit in top_subreddits:
    try:
        downloaded_count = fetch_subreddit(subreddit, 20, target_base_directory='top_subreddits_data', print_frequency = 5, max_runtime = 6000)
        meta_data["subreddit"].append(subreddit)
        meta_data["total_downloaded_posts"].append(downloaded_count)
    except Exception as e:
        exception_counter += 1

        exceptions_data["exception_type"].append(type(e).__name__)
        exceptions_data["exception_message"].append(str(e))
        exceptions_data["subreddit"].append(subreddit)

        if exception_counter % 3 == 0: print(f'{exception_counter} exceptions so far. Latest: {str(e)}, at subreddit = {subreddit}')

# save download numbers log

meta_data_df = pd.DataFrame(meta_data)
timestamp_str = scrape_start_time.strftime('%Y-%m-%d_%H-%M-%S')
filename = f"meta_data_{timestamp_str}.csv"
meta_data_df.to_csv(filename, index=False)

#if there actually were some exceptions, save the log:
if exception_counter: 
    exceptions_df = pd.DataFrame(exceptions_data)
    timestamp_str = scrape_start_time.strftime('%Y-%m-%d_%H-%M-%S')
    filename = f"exceptions_{timestamp_str}.csv"
    exceptions_df.to_csv(filename, index=False)
else:
    print("No exceptions raised!")

Downloaded 1 posts from Home so far (time 12:37).
Downloaded 6 posts from Home so far (time 12:37).
Downloaded 11 posts from Home so far (time 12:37).
Downloaded 16 posts from Home so far (time 12:37).
Downloaded 20 posts from Home in total (time 12:37).
Downloaded 1 posts from AskReddit so far (time 12:38).
Downloaded 6 posts from AskReddit so far (time 12:41).
Sleeping for 21.00 seconds until rate limit reset (time 12:49:53).
Resumed: remaining requests = 999.0. (time 12:50:14)
Stopping fetch for AskReddit, exceeded max runtime.
Downloaded 9 posts from AskReddit in total (time 12:59).
Sleeping for 21.00 seconds until rate limit reset (time 12:59:53).
Resumed: remaining requests = 999.0. (time 13:00:14)
Downloaded 1 posts from NoStupidQuestions so far (time 13:03).
Downloaded 6 posts from NoStupidQuestions so far (time 13:04).
Downloaded 11 posts from NoStupidQuestions so far (time 13:05).
Downloaded 16 posts from NoStupidQuestions so far (time 13:05).
Downloaded 20 posts from NoStupi